In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: RISKY OR SAFE)

In [23]:
tickers_df = pd.read_csv("Tickers_Example.csv", header=None).rename(columns={0:'Ticker'})
tickers_df

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC


In [24]:
# Create an info cell for each ticker, saving time when filtering the tickers by volume and country.
info_column = {}
ticker_obj_lst = []

# Add info to dict
for idx, row in tickers_df.iterrows():
    ticker = row[0] # Get ticker name
    ticker = yf.Ticker(ticker)
    info_column[idx] = ticker.fast_info
    print('.', end='') # So we can tell if the code is running

# Create column for the info]
tickers_df['Info'] = pd.Series(info_column)
tickers_df

........................................

,Ticker,Info
0,AAPL,"lazy-loading dict with keys = ['currency', 'da..."
1,ABBV,"lazy-loading dict with keys = ['currency', 'da..."
2,ABT,"lazy-loading dict with keys = ['currency', 'da..."
3,ACN,"lazy-loading dict with keys = ['currency', 'da..."
4,AGN,"lazy-loading dict with keys = ['currency', 'da..."
5,AIG,"lazy-loading dict with keys = ['currency', 'da..."
6,AMZN,"lazy-loading dict with keys = ['currency', 'da..."
7,AXP,"lazy-loading dict with keys = ['currency', 'da..."
8,BA,"lazy-loading dict with keys = ['currency', 'da..."
9,BAC,"lazy-loading dict with keys = ['currency', 'da..."


In [32]:
tickers_df.iloc[1]['Info']['currency']

'USD'

In [37]:
tickers = []
tickers_hist = {}
tickers_long_hist = {}
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
start_date = "2023-01-01"
end_date = "2023-10-31"
long_start_date = "2018-10-31"
threshold_day = 18
days_in_month = []

CAD_stocks = []
USD_stocks = []

for ticker in tickers_df['Ticker']:
    currTicker = yf.Ticker(ticker)
    currHist = currTicker.history(start=start_date, end=end_date)
    currFastInfo = currTicker.fast_info

    if currHist.empty:
        print(ticker, "is delisted.")
    elif ticker in tickers:
        print(ticker, "is a duplicate.")
    else:
        tickers_hist[ticker] = currHist
        tickers_long_hist[ticker] = currTicker.history(start=long_start_date, end=end_date)

        group_months = currHist.reset_index().groupby(pd.Grouper(key='Date', freq='M'))
        dfs = [group for _, group in group_months]
        filtered_dfs = []
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))
                filtered_dfs.append(df)
        currHist_updated = pd.concat(filtered_dfs)
        currHist_updated.set_index("Date", inplace=True)

        if currFastInfo["currency"] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif currHist["Volume"].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", currFastInfo["lastVolume"], "and average days of", np.mean(days_in_month))
        else:
            tickers.append(ticker)
            if currFastInfo['currency'] == 'USD':
                USD_stocks.append(ticker)
            elif currFastInfo['currency'] == 'CAD':
                CAD_stocks.append(ticker)
                
print("\nValid CAD Stocks:\n", CAD_stocks)
print("\nValid USD Stocks:\n", USD_stocks)

AGN: No timezone found, symbol may be delisted


AGN is delisted.


CELG: No timezone found, symbol may be delisted


CELG is delisted.


MON: No timezone found, symbol may be delisted


MON is delisted.


RTN: No timezone found, symbol may be delisted


RTN is delisted.

Valid CAD Stocks:
 ['RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO']

Valid USD Stocks:
 ['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.